In [1]:
# !pip install flask_cors
# !pip install flask
# !pip install waitress

In [2]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from flask import Flask
from flask import jsonify, request
from flask_cors import CORS

In [9]:
def load_model_file(filename):
    obj = pickle.load(open(filename, 'rb'))
    print(obj)
    
    model = obj['model']
    vectorizer = obj['vectorizer']
    
    return model, vectorizer

def predict(sentences, model, vectorizer):
    sentences = sentences if isinstance(sentences, list) else [sentences]
    vector = vectorizer.transform(sentences)
    return model.predict(vector)

In [10]:
model, vectorizer = load_model_file('movie_model.pkl')

{'model': LogisticRegression(solver='liblinear'), 'vectorizer': TfidfVectorizer(max_features=3000, stop_words='english')}


In [5]:
print(predict('this is a great movie.', model, vectorizer))

['positive']


In [6]:
print(predict('this is a horrible movie.', model, vectorizer))

['negative']


In [7]:
app = Flask(__name__)
CORS(app)

@app.route('/')
def start():
    sentence = request.args.get('sentence')

    if sentence == None: 
        return jsonify({})

    # print(sentence)
    
    result = predict(sentence, model, vectorizer)
    
    # print(result)
    return jsonify({
        'sentence': sentence,
        'sentiment': result[0]
    })

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2022 19:32:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2022 19:33:11] "GET /?sentence=fun%20movie HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2022 19:33:59] "GET /?sentence=movie%20was%20great HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2022 19:34:22] "GET /?sentence=movie%20was%20so%20horrible%20I%20fell%20asleep HTTP/1.1" 200 -


## For production

In [ ]:
from flask import Flask
from flask import jsonify, request
from flask_cors import CORS
from waitress import serve

app = Flask(__name__)
CORS(app)

@app.route('/')
def start():
    sentence = request.args.get('sentence')
    
    if sentence == None: 
        return jsonify({})

    # print(sentence)
    
    result = predict(sentence, model, vectorizer)
    
    # print(result)
    return jsonify({
        'sentence': sentence,
        'sentiment': result[0]
    })

if __name__ == '__main__':
    serve(app, host="0.0.0.0", port=80)

In [8]:
!ls

IMDB Dataset.csv      movie_model.pkl       setting_up_server.txt
consuming_api.ipynb   server.ipynb          user-data.txt
ml_training.ipynb     server.py


In [ ]:
!aws s3 cp movie_model.pkl s3://com.msarica.ds/movie_model.pkl
!aws s3 cp server.py s3://com.msarica.ds/server.py